In [3]:
import os
import numpy as np
from itertools import combinations

class UltrametricTree:
    def __init__(self, n):
        self.n = n
        self.adj_matrix = np.zeros((2 * n - 1, 2 * n - 1))
        self.distances = np.zeros((2 * n - 1, 2 * n - 1))
        self.node_count = n

    def add_edge(self, i, j, weight):
        self.adj_matrix[i][j] = weight
        self.adj_matrix[j][i] = weight

    def update_distances(self):
        self.distances = np.copy(self.adj_matrix)
        for k in range(self.node_count):
            for i in range(self.node_count):
                for j in range(self.node_count):
                    if self.distances[i][j] > self.distances[i][k] + self.distances[k][j]:
                        self.distances[i][j] = self.distances[i][k] + self.distances[k][j]
        
    def compute_weight(self):
        return np.sum(self.adj_matrix) / 2

    def is_ultrametric(self):
        for i in range(self.node_count):
            for j in range(self.node_count):
                for k in range(self.node_count):
                    if self.distances[i][j] > max(self.distances[i][k], self.distances[k][j]):
                        return False

        root = 2 * self.n - 2
        leaf_distances = [self.distances[root][i] for i in range(self.n)]
        if len(set(leaf_distances)) != 1:
            return False

        return True

def generate_combinations(n):
    return list(combinations(range(n), 2))

def merge_clusters(clusters, merge_a, merge_b):
    new_clusters = clusters[:]
    new_cluster = clusters[merge_a] + clusters[merge_b]
    new_clusters.pop(max(merge_a, merge_b))
    new_clusters.pop(min(merge_a, merge_b))
    new_clusters.append(new_cluster)
    return new_clusters

def brute_force_ultrametric_tree(distance_matrix):
    n = len(distance_matrix)
    all_combinations = generate_combinations(n)
    best_tree = None
    min_weight = float('inf')
    
    def recursive_build_tree(clusters, heights, current_tree, current_weight):
        nonlocal best_tree, min_weight
        
        if len(clusters) == 1:
            if current_weight < min_weight:
                min_weight = current_weight
                best_tree = current_tree
            return
        
        for merge_a, merge_b in generate_combinations(len(clusters)):
            new_clusters = merge_clusters(clusters, merge_a, merge_b)
            new_node = len(clusters) + n - len(new_clusters) - 1
            new_height = max(distance_matrix[a][b] for a in clusters[merge_a] for b in clusters[merge_b]) / 2
            
            new_tree = UltrametricTree(n)
            new_tree.adj_matrix[:current_tree.node_count, :current_tree.node_count] = current_tree.adj_matrix[:current_tree.node_count, :current_tree.node_count]
            new_tree.node_count = current_tree.node_count + 1
            
            for node in clusters[merge_a] + clusters[merge_b]:
                new_tree.add_edge(new_node, node, new_height - heights[node])
                
            new_heights = heights[:]
            new_heights[new_node] = new_height
            
            new_weight = current_weight + (new_height * len(new_clusters[-1]))
            recursive_build_tree(new_clusters, new_heights, new_tree, new_weight)
    
    initial_clusters = [[i] for i in range(n)]
    initial_heights = [0] * (2 * n - 1)
    initial_tree = UltrametricTree(n)
    
    recursive_build_tree(initial_clusters, initial_heights, initial_tree, 0)
    
    best_tree.update_distances()
    return best_tree if best_tree.is_ultrametric() else None

def load_distance_matrix(file_path):
    return np.loadtxt(file_path, delimiter=' ')

def load_all_distance_matrices(directory_path):
    matrices = {}
    for file_name in os.listdir(directory_path):
        if file_name.startswith('matrix') and file_name.endswith('.txt'):
            file_path = os.path.join(directory_path, file_name)
            matrices[file_name] = load_distance_matrix(file_path)
    return matrices

directory_path = 'tests/'
distance_matrices = load_all_distance_matrices(directory_path)

for file_name, distance_matrix in distance_matrices.items():
    print(f"Processing matrix from file: {file_name}")
    optimal_tree = brute_force_ultrametric_tree(distance_matrix)
    if optimal_tree.is_ultrametric():
        print(optimal_tree.adj_matrix)
        print("Weight of the optimal tree:", optimal_tree.compute_weight())
    else:
        print("No ultrametric tree found.")


Processing matrix from file: matrix3.txt
[[0.  0.  0.  0.  3.5 0.  0. ]
 [0.  0.  0.  0.  3.5 0.  0. ]
 [0.  0.  0.  0.  3.5 0.  0. ]
 [0.  0.  0.  0.  3.5 0.  0. ]
 [3.5 3.5 3.5 3.5 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0. ]]
Weight of the optimal tree: 14.0
Processing matrix from file: matrix8.txt
[[ 0.  0.  0.  0.  0. 10.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 10.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 10.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 10.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 10.  0.  0.  0.]
 [10. 10. 10. 10. 10.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]]
Weight of the optimal tree: 50.0
Processing matrix from file: matrix5.txt
[[0. 0. 0. 0. 0. 5. 0. 0. 0.]
 [0. 0. 0. 0. 0. 5. 0. 0. 0.]
 [0. 0. 0. 0. 0. 5. 0. 0. 0.]
 [0. 0. 0. 0. 0. 5. 0. 0. 0.]
 [0. 0. 0. 0. 0. 5. 0. 0. 0.]
 [5. 5. 5. 5. 5. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 